In [1]:
import math
import time

# prime_checks.py
# Two prime-checking methods and a short demo/benchmark.
# Optimized version reduces work by:
# - handling small primes and even numbers explicitly,
# - testing divisors only up to sqrt(n),
# - skipping all multiples of 2 and 3 by testing only numbers of the form 6k ± 1.
# This moves complexity from O(n) (basic) to roughly O(sqrt(n)) with a ~3x constant improvement.



def is_prime_basic(n: int) -> bool:
    """Naive trial division: test divisibility by every integer 2..n-1.
    Complexity: O(n). Good only for very small n."""
    if n < 2:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True


def is_prime_optimized(n: int) -> bool:
    """Optimized trial division:
    - handle n < 2 and small primes,
    - eliminate even numbers and multiples of 3,
    - test i and i+2 for i = 5, 11, 17, ... (i += 6) while i*i <= n.
    Complexity: O(sqrt(n)) and fewer divisions in practice."""
    if n < 2:
        return False
    if n in (2, 3):
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    # check factors of form 6k-1 and 6k+1
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True


if __name__ == "__main__":
    # quick correctness check
    samples = [1, 2, 3, 4, 17, 18, 19, 91, 97, 100_003]
    for s in samples:
        a = is_prime_basic(s)
        b = is_prime_optimized(s)
        print(f"{s}: basic={a}, optimized={b}")

    # small benchmark to illustrate difference (choose moderate n to keep runtime reasonable)
    n = 100_003  # around 1e5, prime candidate
    for func in (is_prime_basic, is_prime_optimized):
        t0 = time.perf_counter()
        result = func(n)
        dt = time.perf_counter() - t0
        print(f"{func.__name__}: prime={result}, time={dt:.6f}s")

1: basic=False, optimized=False
2: basic=True, optimized=True
3: basic=True, optimized=True
4: basic=False, optimized=False
17: basic=True, optimized=True
18: basic=False, optimized=False
19: basic=True, optimized=True
91: basic=False, optimized=False
97: basic=True, optimized=True
100003: basic=True, optimized=True
is_prime_basic: prime=True, time=0.007119s
is_prime_optimized: prime=True, time=0.000008s


In [3]:
def fib_recursive(n: int) -> int:
    """Simple recursive Fibonacci. Exponential time for large n."""
    if n < 0:
        raise ValueError("n must be non-negative")
    if n < 2:
        return n
    return fib_recursive(n - 1) + fib_recursive(n - 2)


def fib_memo(n: int, _cache={0: 0, 1: 1}) -> int:
    """Recursive Fibonacci with memoization (fast)."""
    if n < 0:
        raise ValueError("n must be non-negative")
    if n in _cache:
        return _cache[n]
    _cache[n] = fib_memo(n - 1, _cache) + fib_memo(n - 2, _cache)
    return _cache[n]


# Print small values using the plain recursive function
print("Plain recursive (n, fib):")
for i in range(11):
    print(i, fib_recursive(i))

# Print a larger value using the memoized version
print("\nMemoized result:")
print("fib_memo(50) =", fib_memo(50))

Plain recursive (n, fib):
0 0
1 1
2 1
3 2
4 3
5 5
6 8
7 13
8 21
9 34
10 55

Memoized result:
fib_memo(50) = 12586269025


In [7]:
# Robust examples demonstrating error handling with clear explanations.
# This cell re-uses functions and variables already defined in the notebook:
# is_prime_basic, is_prime_optimized, fib_recursive, fib_memo, sample_path,
# i, n, s, dt, etc.

# 1) Division by zero demonstration
try:
    # i is 10 in the notebook; this will raise ZeroDivisionError.
    division_result = dt / (i - 10)
    print("Division result:", division_result)
except ZeroDivisionError as exc:
    # Clear explanation for this exception
    print("ZeroDivisionError: attempted to divide by zero. "
          "Check the divisor expression (i - 10). Exception detail:", exc)

# 2) Fibonacci function error handling
# Valid call
try:
    print("fib_recursive(i) ->", fib_recursive(i))
except (ValueError, RecursionError) as exc:
    print("Error calling fib_recursive(i):", exc)
except Exception as exc:
    print("Unexpected error calling fib_recursive(i):", exc)

# Intentional invalid input to demonstrate ValueError handling
try:
    print("fib_recursive(-5) ->", fib_recursive(-5))
except ValueError as exc:
    print("ValueError: fib_recursive requires a non-negative integer. "
          "Provide n >= 0. Exception detail:", exc)

# Intentional large input to demonstrate RecursionError handling
try:
    # Large value likely to exceed recursion depth and raise RecursionError
    fib_recursive(1000)
except RecursionError as exc:
    print("RecursionError: recursive implementation exceeded maximum recursion depth. "
          "Use fib_memo or an iterative implementation for large n. Exception detail:", exc)

# fib_memo with wrong type to show TypeError (comparison inside will raise it)
try:
    print("fib_memo('10') ->", fib_memo('10'))
except TypeError as exc:
    print("TypeError: fib_memo expects an integer input. Convert or validate the input first. Exception detail:", exc)
except Exception as exc:
    print("Unexpected error calling fib_memo with wrong type:", exc)

# 3) Safe wrapper for prime checks that validates input type and reports errors
def safe_is_prime(func, value):
    try:
        # explicit type check: prime checkers expect int
        if not isinstance(value, int):
            raise TypeError(f"Input must be int, got {type(value).__name__}")
        result = func(value)
        print(f"{func.__name__}({value}) -> {result}")
    except TypeError as exc:
        print(f"TypeError in {func.__name__}: {exc}")
    except Exception as exc:
        print(f"Unexpected error in {func.__name__}: {exc}")

# Valid usage
safe_is_prime(is_prime_optimized, n)   # n is 100003 in the notebook
# Invalid usage to show type validation
safe_is_prime(is_prime_basic, 10.5)

# 4) File I/O error handling: attempt to read sample_path
try:
    with open(sample_path, "r", encoding="utf-8") as fh:
        preview = fh.read(1024)
        print(f"Read {len(preview)} characters from {sample_path!r}")
except FileNotFoundError as exc:
    print("FileNotFoundError: the file specified by sample_path was not found. "
          "Provide a correct path or ensure the file exists. Exception detail:", exc)
except PermissionError as exc:
    print("PermissionError: insufficient permission to read the file. "
          "Adjust file permissions or choose a different path. Exception detail:", exc)
except Exception as exc:
    print("Unexpected error while reading file:", exc)

# 5) Handling user cancellation / long-running operations
try:
    print("Starting a potentially long computation with is_prime_basic(20000). "
          "Use Ctrl+C to cancel and trigger KeyboardInterrupt handling.")
    # use a moderately large number that could be slow but usually finishes fast enough here
    long_result = is_prime_basic(20000)
    print("is_prime_basic(20000) completed:", long_result)
except KeyboardInterrupt:
    print("KeyboardInterrupt: computation was cancelled by the user. "
          "Consider using the optimized prime checker (is_prime_optimized) instead.")
except Exception as exc:
    print("Unexpected error during long computation:", exc)# Robust examples demonstrating error handling with clear explanations.
# This cell re-uses functions and variables already defined in the notebook:
# is_prime_basic, is_prime_optimized, fib_recursive, fib_memo, sample_path,
# i, n, s, dt, etc.

# 1) Division by zero demonstration
try:
    # i is 10 in the notebook; this will raise ZeroDivisionError.
    division_result = dt / (i - 10)
    print("Division result:", division_result)
except ZeroDivisionError as exc:
    # Clear explanation for this exception
    print("ZeroDivisionError: attempted to divide by zero. "
          "Check the divisor expression (i - 10). Exception detail:", exc)

# 2) Fibonacci function error handling
# Valid call
try:
    print("fib_recursive(i) ->", fib_recursive(i))
except (ValueError, RecursionError) as exc:
    print("Error calling fib_recursive(i):", exc)
except Exception as exc:
    print("Unexpected error calling fib_recursive(i):", exc)

# Intentional invalid input to demonstrate ValueError handling
try:
    print("fib_recursive(-5) ->", fib_recursive(-5))
except ValueError as exc:
    print("ValueError: fib_recursive requires a non-negative integer. "
          "Provide n >= 0. Exception detail:", exc)

# Intentional large input to demonstrate RecursionError handling
try:
    # Large value likely to exceed recursion depth and raise RecursionError
    fib_recursive(1000)
except RecursionError as exc:
    print("RecursionError: recursive implementation exceeded maximum recursion depth. "
          "Use fib_memo or an iterative implementation for large n. Exception detail:", exc)

# fib_memo with wrong type to show TypeError (comparison inside will raise it)
try:
    print("fib_memo('10') ->", fib_memo('10'))
except TypeError as exc:
    print("TypeError: fib_memo expects an integer input. Convert or validate the input first. Exception detail:", exc)
except Exception as exc:
    print("Unexpected error calling fib_memo with wrong type:", exc)

# 3) Safe wrapper for prime checks that validates input type and reports errors
def safe_is_prime(func, value):
    try:
        # explicit type check: prime checkers expect int
        if not isinstance(value, int):
            raise TypeError(f"Input must be int, got {type(value).__name__}")
        result = func(value)
        print(f"{func.__name__}({value}) -> {result}")
    except TypeError as exc:
        print(f"TypeError in {func.__name__}: {exc}")
    except Exception as exc:
        print(f"Unexpected error in {func.__name__}: {exc}")

# Valid usage
safe_is_prime(is_prime_optimized, n)   # n is 100003 in the notebook
# Invalid usage to show type validation
safe_is_prime(is_prime_basic, 10.5)

# 4) File I/O error handling: attempt to read sample_path
try:
    with open(sample_path, "r", encoding="utf-8") as fh:
        preview = fh.read(1024)
        print(f"Read {len(preview)} characters from {sample_path!r}")
except FileNotFoundError as exc:
    print("FileNotFoundError: the file specified by sample_path was not found. "
          "Provide a correct path or ensure the file exists. Exception detail:", exc)
except PermissionError as exc:
    print("PermissionError: insufficient permission to read the file. "
          "Adjust file permissions or choose a different path. Exception detail:", exc)
except Exception as exc:
    print("Unexpected error while reading file:", exc)

# 5) Handling user cancellation / long-running operations
try:
    print("Starting a potentially long computation with is_prime_basic(20000). "
          "Use Ctrl+C to cancel and trigger KeyboardInterrupt handling.")
    # use a moderately large number that could be slow but usually finishes fast enough here
    long_result = is_prime_basic(20000)
    print("is_prime_basic(20000) completed:", long_result)
except KeyboardInterrupt:
    print("KeyboardInterrupt: computation was cancelled by the user. "
          "Consider using the optimized prime checker (is_prime_optimized) instead.")
except Exception as exc:
    print("Unexpected error during long computation:", exc)# Robust examples demonstrating error handling with clear explanations.
# This cell re-uses functions and variables already defined in the notebook:
# is_prime_basic, is_prime_optimized, fib_recursive, fib_memo, sample_path,
# i, n, s, dt, etc.

# 1) Division by zero demonstration
try:
    # i is 10 in the notebook; this will raise ZeroDivisionError.
    division_result = dt / (i - 10)
    print("Division result:", division_result)
except ZeroDivisionError as exc:
    # Clear explanation for this exception
    print("ZeroDivisionError: attempted to divide by zero. "
          "Check the divisor expression (i - 10). Exception detail:", exc)

# 2) Fibonacci function error handling
# Valid call
try:
    print("fib_recursive(i) ->", fib_recursive(i))
except (ValueError, RecursionError) as exc:
    print("Error calling fib_recursive(i):", exc)
except Exception as exc:
    print("Unexpected error calling fib_recursive(i):", exc)

# Intentional invalid input to demonstrate ValueError handling
try:
    print("fib_recursive(-5) ->", fib_recursive(-5))
except ValueError as exc:
    print("ValueError: fib_recursive requires a non-negative integer. "
          "Provide n >= 0. Exception detail:", exc)

# Intentional large input to demonstrate RecursionError handling
try:
    # Large value likely to exceed recursion depth and raise RecursionError
    fib_recursive(1000)
except RecursionError as exc:
    print("RecursionError: recursive implementation exceeded maximum recursion depth. "
          "Use fib_memo or an iterative implementation for large n. Exception detail:", exc)

# fib_memo with wrong type to show TypeError (comparison inside will raise it)
try:
    print("fib_memo('10') ->", fib_memo('10'))
except TypeError as exc:
    print("TypeError: fib_memo expects an integer input. Convert or validate the input first. Exception detail:", exc)
except Exception as exc:
    print("Unexpected error calling fib_memo with wrong type:", exc)

# 3) Safe wrapper for prime checks that validates input type and reports errors
def safe_is_prime(func, value):
    try:
        # explicit type check: prime checkers expect int
        if not isinstance(value, int):
            raise TypeError(f"Input must be int, got {type(value).__name__}")
        result = func(value)
        print(f"{func.__name__}({value}) -> {result}")
    except TypeError as exc:
        print(f"TypeError in {func.__name__}: {exc}")
    except Exception as exc:
        print(f"Unexpected error in {func.__name__}: {exc}")

# Valid usage
safe_is_prime(is_prime_optimized, n)   # n is 100003 in the notebook
# Invalid usage to show type validation
safe_is_prime(is_prime_basic, 10.5)

# 4) File I/O error handling: attempt to read sample_path
try:
    with open(sample_path, "r", encoding="utf-8") as fh:
        preview = fh.read(1024)
        print(f"Read {len(preview)} characters from {sample_path!r}")
except FileNotFoundError as exc:
    print("FileNotFoundError: the file specified by sample_path was not found. "
          "Provide a correct path or ensure the file exists. Exception detail:", exc)
except PermissionError as exc:
    print("PermissionError: insufficient permission to read the file. "
          "Adjust file permissions or choose a different path. Exception detail:", exc)
except Exception as exc:
    print("Unexpected error while reading file:", exc)

# 5) Handling user cancellation / long-running operations
try:
    print("Starting a potentially long computation with is_prime_basic(20000). "
          "Use Ctrl+C to cancel and trigger KeyboardInterrupt handling.")
    # use a moderately large number that could be slow but usually finishes fast enough here
    long_result = is_prime_basic(20000)
    print("is_prime_basic(20000) completed:", long_result)
except KeyboardInterrupt:
    print("KeyboardInterrupt: computation was cancelled by the user. "
          "Consider using the optimized prime checker (is_prime_optimized) instead.")
except Exception as exc:
    print("Unexpected error during long computation:", exc)

ZeroDivisionError: attempted to divide by zero. Check the divisor expression (i - 10). Exception detail: float division by zero
fib_recursive(i) -> 55
ValueError: fib_recursive requires a non-negative integer. Provide n >= 0. Exception detail: n must be non-negative


KeyboardInterrupt: 

In [8]:
from typing import Dict
from getpass import getpass

# simple_login.py
# Minimal AI-style generated login system (educational, intentionally simple)

# In-memory user store (username -> password in plain text)
USERS: Dict[str, str] = {}

def register(username: str, password: str) -> bool:
    """Register a new user. Returns True on success, False if user exists or invalid input."""
    username = username.strip()
    if not username or not password:
        return False
    if username in USERS:
        return False
    # NOTE: password stored directly (plain-text) — see analysis below
    USERS[username] = password
    return True

def verify_login(username: str, password: str) -> bool:
    """Verify credentials. Returns True if they match."""
    stored = USERS.get(username)
    if stored is None:
        return False
    # direct equality check used for verification
    return stored == password

def demo_interactive():
    print("=== Register ===")
    u = input("username: ")
    p = getpass("password: ")
    if register(u, p):
        print("Registered.")
    else:
        print("Registration failed (empty username/password or already exists).")

    print("\n=== Login ===")
    u2 = input("username: ")
    p2 = getpass("password: ")
    if verify_login(u2, p2):
        print("Login successful.")
    else:
        print("Login failed.")

if __name__ == "__main__":
    # tiny demo of usage
    demo_interactive()

=== Register ===
Registered.

=== Login ===
Login successful.


In [9]:
import logging
import datetime
import hashlib
import secrets
import re

"""
activity_logger.py -- Example of logging user login activity, analysis, and a privacy-preserving variant.

Analysis (what is sensitive / why risky):
- Logged data in a naive system: username, full IP address, timestamp.
- Why risky:
    * Username identifies a real person (personal data).
    * Full IP address is quasi-identifying and can be used to geolocate or correlate activity.
    * Timestamps combined with identifiers allow building behavioral profiles.
    * Storing raw identifiers increases impact of a data breach or unauthorized access.

Mitigations implemented below:
- Do not log raw username; log a one-way pseudonym (hashed with a per-session salt).
- Do not log full IP; anonymize it (zero-out last IPv4 octet or compress IPv6).
- Keep timestamp (UTC) but do not store extra contextual metadata.
"""


# Per-process salt: makes pseudonyms non-reversible across different runs.
# (If you need correlation across restarts, use a stable secret stored securely.)
_SALT = secrets.token_hex(16)

# logger setup
logger = logging.getLogger("activity")
if not logger.handlers:
        handler = logging.StreamHandler()
        handler.setFormatter(logging.Formatter("%(asctime)s %(levelname)s %(message)s"))
        logger.addHandler(handler)
logger.setLevel(logging.INFO)


def naive_log_login(username: str, ip: str):
        """Naive logger that writes raw username and IP (illustrative; not recommended)."""
        ts = datetime.datetime.utcnow().isoformat() + "Z"
        logger.info(f"login username={username!r} ip={ip!r} timestamp={ts}")


def anonymize_ip(ip: str) -> str:
        """Anonymize an IP address.
        - IPv4: zero last octet (e.g. 203.0.113.45 -> 203.0.113.0)
        - IPv6: keep first 4 hextets then '::' (simple truncation to reduce identifiability)
        - Fallback: return '<anon-ip>'
        """
        # IPv4
        m = re.match(r"^(\d{1,3}\.\d{1,3}\.\d{1,3})\.\d{1,3}$", ip)
        if m:
                return f"{m.group(1)}.0"
        # IPv6 (very simple heuristic)
        if ":" in ip:
                parts = ip.split(":")
                if len(parts) >= 4:
                        return ":".join(parts[:4]) + "::"
                return ip.split("::", 1)[0] + "::"
        return "<anon-ip>"


def pseudonymize_username(username: str) -> str:
        """One-way pseudonym: SHA-256 of (salt + username), truncated for brevity."""
        h = hashlib.sha256((_SALT + username).encode("utf-8")).hexdigest()
        return f"user-{h[:12]}"


def privacy_preserving_log_login(username: str, ip: str):
        """Log an event with reduced and anonymized data."""
        ts = datetime.datetime.utcnow().isoformat() + "Z"
        pseudonym = pseudonymize_username(username)
        anon_ip = anonymize_ip(ip)
        # Minimal structured message: do not include raw username or raw IP
        logger.info(f"login pseudonym={pseudonym} ip={anon_ip} timestamp={ts}")


# --- Example / demo usage ---
if __name__ == "__main__":
        # Example raw data (in real usage these come from the web framework / auth layer)
        example_username = "chaithanya"
        example_ip_v4 = "203.0.113.45"
        example_ip_v6 = "2001:0db8:85a3:0000:0000:8a2e:0370:7334"

        # Demonstrate naive logging (for contrast) -- avoid enabling in production
        naive_log_login(example_username, example_ip_v4)

        # Recommended: privacy-preserving logging
        privacy_preserving_log_login(example_username, example_ip_v4)
        privacy_preserving_log_login(example_username, example_ip_v6)

        # If you want to correlate events in the same run, the pseudonym above is stable.
        # To correlate across runs, use a securely stored salt or a keyed HMAC with a server secret.

/var/folders/z1/7q_jld1s7l1cplbwjr4n4tc40000gn/T/ipykernel_7882/2666176703.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.datetime.utcnow().isoformat() + "Z"
2026-01-30 21:47:16,755 INFO login username='chaithanya' ip='203.0.113.45' timestamp=2026-01-30T16:17:16.755354Z
/var/folders/z1/7q_jld1s7l1cplbwjr4n4tc40000gn/T/ipykernel_7882/2666176703.py:71: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.datetime.utcnow().isoformat() + "Z"
2026-01-30 21:47:16,758 INFO login pseudonym=user-27e2576667a8 ip=203.0.113.0 timestamp=2026-01-30T16:17:16.757213Z
2026-01-30 21:47:16,758 INFO login pseudonym=user-27e2576667a8 ip=2001:0db8:85a3:0000:: timestamp